## Model Setup

#### Detect current folder to avoid package import error

In [3]:
import os, sys
currentdir = os.path.dirname(os.path.realpath(''))
parentdir = os.path.dirname(currentdir)
sys.path.append(currentdir)

#### Import Opensource Packages

In [4]:
import numpy as np
import pandas as pd

import idaes
import pyomo.environ as pyo

ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

## Create Concrete Model

In [ ]:
from pyomo.environ import *

model = ConcreteModel(name = "Price Optimization")

#### Data Import

In [ ]:
import mydata_V2_0
Data_dict = mydata_V2_0.Data_dict

#### Define Model Sets

In [ ]:
model.I = Set(initialize=Data_dict['countries'])
model.J = Set(initialize=Data_dict['sites'])
model.T = Set(initialize=Data_dict['time_periods'])

### Define Decision Variables

In [ ]:
model.p = Var(
    model.I,
    model.J, 
    model.T, 
    domain=NonNegativeReals,
    doc = 'dv: price per kWh'
)

### Define Parameters

In [ ]:
model.e = Param(model.I, model.T, initialize=Data_dict['cost_energy'], within=NonNegativeReals)
model.r = Param(model.I, model.T, initialize=Data_dict['cost_revshare'], within=NonNegativeReals)

model.p_max = Param(model.I, model.J, model.T, initialize=Data_dict['p_max'], within=NonNegativeReals)

model.d_0 = Param(model.I, model.J, model.T, initialize=Data_dict['d_0'], within=NonNegativeReals)
model.d_p = Param(model.I, model.J, model.T, initialize=Data_dict['d_p'], within=NonNegativeReals)
model.d_max = Param(model.I, model.J, model.T, initialize=Data_dict['d_max'], within=NonNegativeReals)

def demand(model, i, j, t):
    '''
    Calculates the demand depending on the price for each country, site, and time period.

    This function implements a linear demand model where `d_0` is the intercept and `d_p` is the slope of the demand function. 
    The demand is inversely related to the price.

    Args:
        model: The model object containing parameters and variables.
        i: The index representing the country.
        j: The index representing the site.
        t: The time period index.

    Returns:
        float: The calculated demand in kWh for the specified country, site, and time period.
        
    '''
    return model.d_0[i,j,t] - model.d_p[i,j,t] * model.p[i,j,t]
model.D = Expression(model.I, model.J, model.T, rule=demand) 

def total_cost_variable(model, i, j, t):
    '''
    Calculates the total variable cost for each country, site, and time period.

    The total variable cost includes the sum of energy cost and revenue share cost.

    Args:
        model: The model object containing parameters and variables.
        i: The index representing the country.
        j: The index representing the site.
        t: The time period index.

    Returns:
        float: The total variable cost in € for the specified country, site, and time period.
        
    '''
    return (
        model.e[i,t] * model.D[i,j,t] +
        model.r[i,t] * model.p[i,j,t] * model.D[i,j,t]
        )
model.CV = Expression(model.I, model.J, model.T, rule=total_cost_variable)

### Define Objective Function

In [ ]:
def operational_profit(model):
    '''
    
    Calculates the operational profit for each country, site, and time period.
    
    The operational profit is calculated as the difference between the revenue (price multiplied by demand)
    and the total variable costs, summed over all countries, sites, and time periods.
    
    Args:
        model: The model object that contains parameters and variables.
        i: The index representing the country.
        j: The index representing the site.
        t: The time period index.
        
    Returns:
        float: The total operational profit in € across all countries, sites, and time periods.
        
    '''
    return sum(
        model.D[i,j,t] * model.p[i,j,t] - model.CV[i,j,t]
        for i in model.I for j in model.J for t in model.T 
    )
model.profit = Objective(rule=operational_profit, sense=maximize)

### Define Constraints

In [ ]:
def minimum_price(model, i, j, t):
    '''
    
    Defines a constraint ensuring the optimal price for each country, site, and time period is 
    not lower than the total variable cost.
    
    This function calculates the variable cost for a given country, site, and time period, then imposes a 
    constraint that the price (`model.p`) must be greater than or equal to the variable cost.
    
    Args:
        model: The model object containing parameters and variables.
        i: The index representing the country.
        j: The index representing the site.
        t: The time period index.
        
    Returns:
        A Pyomo constraint expression that ensures the price per kWh is not lower than the total variable cost 
        for each country, site, and time period.
        
    '''
    variable_cost = model.e[i,t] + model.r[i,t] * model.p[i,j,t]
    return model.p[i,j,t] >= variable_cost
model.price_constraint = Constraint(model.I, model.J, model.T, rule=minimum_price)

def maximum_price(model, i, j, t):
    '''
    
    Defines a constraint that ensures the optimal price for each country, site, and time period
    does not exceed a predefined maximum price.
    
    This constraint is important for keeping the price competitive and within market limits.
    It uses the model's price variable (`model.p`) and compares it against the maximum allowable price
    (`model.p_max`) for each country, site, and time period.
    
    Args:
        model: The model object containing parameters and variables.
        i: The index representing the country.
        j: The index representing the site.
        t: The time period index.
        
    Returns:
        A Pyomo constraint expression that ensures the price per kWh does not exceed the maximum allowable
        price for each country, site and time period.
        
    '''
    return model.p[i,j,t] <= model.p_max[i,j,t]
model.price_constraint_max = Constraint(model.I, model.J, model.T, rule=maximum_price)

def maximum_demand(model, i, j, t):
    '''
    
    Defines a constraint ensuring that the demand for each country, site, and time period
    does not exceed a specified maximum demand.
    
    It compares the demand variable (`model.D`) against the maximum allowable demand (`model.d_max`) for each country, site, and time period.
    
    Args:
        model: The model object containing parameters and variables.
        i: The index representing the country.
        j: The index representing the site.
        t: The time period index.
        
    Returns:
        A Pyomo constraint expression that ensures demand in kWh does not exceed the maximum allowable
        demand for each country, site, and time period.
        
    '''
    return model.D[i,j,t] <= model.d_max[i,j,t]
model.demand_constraints = Constraint(model.I, model.J, model.T, rule=maximum_demand)

## Solving the Model

### Optional: Pyomo Configuration  

##### Telling Pyomo where the CPLEX executable is located on the drive by setting the environment variable PYOMO_EXE to point to the CPLEX executable & checking the Pyomo configuration

In [ ]:
import os
os.environ["PYOMO_EXE"] = "/Applications/CPLEX_Studio2211/cplex/bin/x86-64_osx/cplex" # Insert respective path to cplex executable   
print(os.environ.get("PYOMO_EXE"))

### Solver

In [ ]:
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

# Create a solver instance
solver = SolverFactory('cplex', executable="/Applications/CPLEX_Studio2211/cplex/bin/x86-64_osx/cplex") # Insert respective path to cplex executable

# Solve the model
result = solver.solve(model)

# Check the solver status
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    # The model was solved to optimality
    print("Optimal solution found.")
    # Access the variable values
    for var in model.component_data_objects(Var):
        print(f"{var.name}: {var.value}")
else:
    # The model failed to solve
    print("Solver did not find an optimal solution.")

### Display Further Results

In [ ]:
model.D.display()

In [ ]:
model.CV.display()

In [ ]:
model.profit.display()

In [ ]:
model.d_max.display()

In [ ]:
model.p_max.display()